In [1]:
import requests
import time
import csv
import cloudscraper

In [2]:
session = requests.Session()

scraper = cloudscraper.create_scraper()
# Подставь актуальные значения отсюда:
CSRF_TOKEN = "vRRZoscikAodQQLqmpXnTXurcBGFZinJLuzi1mU3"  # <--- ОБНОВИ ЕСЛИ ЧТО
MAX_PAGES = 50  # или сколько хочешь
CATEGORIES = [81005, 81006, 81007, 81009]
BOOK_STATUS = [1, 2]
ORDER_BY = [5]
OUT_FILE = "webnovel_books_full.csv"

session = requests.Session()

headers = {
    "authority": "www.webnovel.com",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "ru,en;q=0.9,ja;q=0.8,cy;q=0.7",
    "cache-control": "no-cache",
    "dnt": "1",
    "pragma": "no-cache",
    "priority": "u=0, i",
    "sec-ch-ua": '"Chromium";v="136", "YaBrowser";v="25.6", "Not.A/Brand";v="99", "Yowser";v="2.5"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "cross-site",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 YaBrowser/25.6.0.0 Safari/537.36"
}

cookies = {
    "bookCitysex": "1",
    "show_lib_tip": "1",
    "webnovel-content-language": "en",
    "webnovel-language": "en",
    "_csrfToken": "vRRZoscikAodQQLqmpXnTXurcBGFZinJLuzi1mU3",
    "NEXT_LOCALE": "en",
    "_ga": "GA1.2.878607571.1665435245",
    "_gid": "GA1.2.1618129538.1755446306",
    "webnovel_uuid": "1743107461_837709012",
    "dontneedgoogleonetap": "1",
    "cf_clearance": "0iweWi5E47yjY4qMjdu72BqJkm882kymvIqd3IP_Yu4-1755451565-1.2.1.1-Ya0zrM0GM1NKXk5jo1EGyGRkjPujd_uFcSDshDf6cffsczweWeR0YilU6SJX1Rq3OgamHYszWTNPBsdQMJ24IGbuwrSNL_Qpt5XRWtOdFfHEpfArxD_oOP2j7aPY6oCC_rR5glOL2nfhaDTU_xFpfwD3WdAoKijL81T0ke134Ku3mQbxUKoyYcp8JhDuWiUf64EngPpMTb9ZAHph95vkM2EXHZzn18s3geZESCH29aw",
    # ... можно добавить остальные cookies при необходимости
}

BASE_API = "https://www.webnovel.com/go/pcm/category/categoryAjax"



In [3]:
def get_books(page, categoryId = 0, bookStatus = 0, orderBy = 1):
    params = {
        "_csrfToken": CSRF_TOKEN,
        "pageIndex": page,
        "categoryId": categoryId,
        "categoryType": 4,
        "bookStatus": bookStatus,
        "orderBy": orderBy
    }

    resp = scraper.get(BASE_API, headers=headers, cookies=cookies, params=params)
    resp.raise_for_status()
    return resp.json().get("data", {}).get("items", [])

def book_to_row(book):
    tags = ", ".join(tag["tagName"] for tag in book.get("tagInfo", []))
    url = f"https://www.webnovel.com/book/{book['bookName'].lower().replace(' ', '-').replace(':', '').replace('?', '')}_{book['bookId']}"
    description = book.get("description", "").replace("\n", " ").replace("\r", " ")
    return [
        book["bookName"],
        book["authorName"],
        book["categoryName"],
        tags,
        book["chapterNum"],
        description,
        url
    ]


In [4]:
# === Сбор всех книг ===
all_books = []
index = 1

for category in CATEGORIES:
    for status in BOOK_STATUS:
        for order in ORDER_BY:
            for page in range(1, MAX_PAGES + 1):
                print(f"[{index}] Загружаю страницу...")
                try:
                    books = get_books(page, category, status, order)
                    if not books:
                        print("Нет больше книг.")
                        break
                    all_books.extend(books)
                except Exception as e:
                    print(f"Ошибка на странице {page}: {e}")
                    break
                index += 1
                time.sleep(15)

print(f"\nСобрано книг: {len(all_books)}")


[1] Загружаю страницу...
[2] Загружаю страницу...
[3] Загружаю страницу...
[4] Загружаю страницу...
[5] Загружаю страницу...
[6] Загружаю страницу...
[7] Загружаю страницу...
[8] Загружаю страницу...
[9] Загружаю страницу...
[10] Загружаю страницу...
[11] Загружаю страницу...
[12] Загружаю страницу...
[13] Загружаю страницу...
[14] Загружаю страницу...
[15] Загружаю страницу...
[16] Загружаю страницу...
[17] Загружаю страницу...
[18] Загружаю страницу...
[19] Загружаю страницу...
[20] Загружаю страницу...
[21] Загружаю страницу...
[22] Загружаю страницу...
[23] Загружаю страницу...
[24] Загружаю страницу...
[25] Загружаю страницу...
[26] Загружаю страницу...
[27] Загружаю страницу...
[28] Загружаю страницу...
[29] Загружаю страницу...
[30] Загружаю страницу...
[31] Загружаю страницу...
[32] Загружаю страницу...
[33] Загружаю страницу...
[34] Загружаю страницу...
[35] Загружаю страницу...
[36] Загружаю страницу...
[37] Загружаю страницу...
[38] Загружаю страницу...
[39] Загружаю страниц

In [5]:
new_books = all_books.copy()

for book in new_books:
    # Добавляем ссылку
    if "bookName" in book and "bookId" in book:
        slug = book["bookName"].lower().replace(" ", "-").replace(":", "").replace("?", "")
        book["url"] = f"https://www.webnovel.com/book/{slug}_{book['bookId']}"
    else:
        book["url"] = ""
        
    if 'tagInfo' in book:
        rpr = []
        for tag_name in book['tagInfo']:
            rpr.append(tag_name['tagName'])
        book['tags'] = ' '.join(rpr)
        
    # Форматируем totalScore
    if "totalScore" in book and isinstance(book.get("totalScore"), (int, float)):
        book["totalScore"] = str(book["totalScore"]).replace(".", ",")

In [6]:
new_books[0]

{'bookId': '28953376100932005',
 'bookName': 'A Search Through Runeterra (Arcane)',
 'description': "Gregori Hastur was a happy man. His wife and his little bundle of joy Annie were all that he needed. But strange incidents and tragedy strike. And so begins his journey to find his daughter while also raising another in the process. Runeterra is dangerous and he? He is even more so. \n\nThis story is Arcane adjacent. So if you have seen the show you can just pick it up and not worry about anything. The characters don't go to Piltover or Zaun until half the story is complete.",
 'categoryId': 81005,
 'categoryName': 'Video Games',
 'authorName': '1Manager1',
 'totalScore': '4,94',
 'translateMode': 0,
 'coverUpdateTime': 1753733215356,
 'tagInfo': [{'id': 41000121,
   'tagName': 'ACTION',
   'enTagName': 'ACTION',
   'likeCount': 0,
   'like': False},
  {'id': 41000120,
   'tagName': 'ADVENTURE',
   'enTagName': 'ADVENTURE',
   'likeCount': 0,
   'like': False},
  {'id': 41000884,
   'ta

In [7]:
import re

# Регулярка для китайских символов
chinese_char_pattern = re.compile(r'[\u4e00-\u9fff]')

# === Формируем список нужных полей ===
excluded_keys = {"description", "tagInfo"}
all_keys = sorted(set().union(*(book.keys() for book in all_books)) - excluded_keys)
all_keys += ["url", "is_chinese"]  # вручную добавим поле URL и is_chinese

with open(OUT_FILE, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=all_keys)
    writer.writeheader()
    for book in all_books:
        row = {
            k: str(book[k]) if k in book and not isinstance(book[k], (str, int, float)) else book.get(k, "")
            for k in all_keys if k not in {"url", "is_chinese"}
        }

        # Добавляем ссылку
        if "bookName" in book and "bookId" in book:
            slug = book["bookName"].lower().replace(" ", "-").replace(":", "").replace("?", "")
            row["url"] = f"https://www.webnovel.com/book/{slug}_{book['bookId']}"
        else:
            row["url"] = ""

        # Форматируем totalScore
        if "totalScore" in row and isinstance(book.get("totalScore"), (int, float)):
            row["totalScore"] = str(book["totalScore"]).replace(".", ",")

        # Проверка на китайские символы
        text_to_check = " ".join(str(v) for v in book.values() if isinstance(v, str))
        row["is_chinese"] = 1 if chinese_char_pattern.search(text_to_check) else 0

        writer.writerow(row)


In [8]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np

# === Пример книги ===
books = all_books

df = pd.DataFrame(books)

# === Регулярка для китайских символов ===
chinese_char_pattern = re.compile(r'[\u4e00-\u9fff]')

# === Проверка на китайские символы ===
def has_chinese(row):
    text = " ".join(str(v) for _, v in row.items() if isinstance(v, str))
    return 1 if chinese_char_pattern.search(text) else 0


df["is_chinese"] = df.apply(has_chinese, axis=1)

# === Объединяем все нужные поля в один текст для кластеризации ===
def combine_text(row):
    tags = " ".join(tag["tagName"] for tag in row.get("tagInfo", []))
    fields = [
        row.get("bookName", ""),
        row.get("description", ""),
        row.get("firstChapterName", ""),
        row.get("categoryName", ""),
        tags
    ]
    combined = " ".join(str(f) for f in fields if f)
    return re.sub(r"[^a-zA-Z0-9\s]", " ", combined).lower()

df["combined_text"] = df.apply(combine_text, axis=1)

# === TF-IDF ===
vectorizer = TfidfVectorizer(stop_words="english", max_features=10000)
X = vectorizer.fit_transform(df["combined_text"])

# === Кластеризация ===
n_clusters = 500
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
df["cluster"] = kmeans.fit_predict(X)

# === Получаем tf-idf матрицу и слова ===
feature_names = vectorizer.get_feature_names_out()

# === Генерация меток кластеров ===
cluster_keywords = {}

for cluster_num in sorted(df["cluster"].unique()):
    cluster_indices = df[df["cluster"] == cluster_num].index
    if not cluster_indices.any():
        continue
    cluster_matrix = X[cluster_indices]
    mean_tfidf = np.asarray(cluster_matrix.mean(axis=0)).flatten()

    top_indices = mean_tfidf.argsort()[::-1][:8]
    top_words = [feature_names[i] for i in top_indices]
    cluster_keywords[cluster_num] = " ".join(top_words)

df["cluster_label"] = df["cluster"].map(cluster_keywords)


In [9]:
# === Сохраняем результат ===
df[["bookId", "bookName", "authorName", "cluster", "cluster_label", "url", "tags", "chapterNum", "totalScore", "firstChapterName", "is_chinese", "publishTime", "coverUpdateTime"]].to_csv("books_clustered_named.csv", index=False)
print("✅ Сохранено с названиями кластеров в books_clustered_named.csv")

✅ Сохранено с названиями кластеров в books_clustered_named.csv


In [10]:
books[0]

{'bookId': '28953376100932005',
 'bookName': 'A Search Through Runeterra (Arcane)',
 'description': "Gregori Hastur was a happy man. His wife and his little bundle of joy Annie were all that he needed. But strange incidents and tragedy strike. And so begins his journey to find his daughter while also raising another in the process. Runeterra is dangerous and he? He is even more so. \n\nThis story is Arcane adjacent. So if you have seen the show you can just pick it up and not worry about anything. The characters don't go to Piltover or Zaun until half the story is complete.",
 'categoryId': 81005,
 'categoryName': 'Video Games',
 'authorName': '1Manager1',
 'totalScore': '4,94',
 'translateMode': 0,
 'coverUpdateTime': 1753733215356,
 'tagInfo': [{'id': 41000121,
   'tagName': 'ACTION',
   'enTagName': 'ACTION',
   'likeCount': 0,
   'like': False},
  {'id': 41000120,
   'tagName': 'ADVENTURE',
   'enTagName': 'ADVENTURE',
   'likeCount': 0,
   'like': False},
  {'id': 41000884,
   'ta